In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
import warnings
warnings.filterwarnings('ignore')

On se propose de procéder à une certaine selection des variables en utilisant des méthodes économétriques de significativité des variables.

L'idée : étudier un problème linéaire de la forme 
$y= \beta_0+ \beta_1x_1 + \beta_2x_2 + \beta_3x_3 + ... + \beta_{num features}x_{num features} + \epsilon$

Puis on test : H_0 : $\beta_i = 0$ pour tous i  

Cele peut nous donner des indications sur quelles variables utiliser 

#### Problème :

Notre target est binaire -> quelles implications ?

## Selection de Features par t-test

In [3]:
import numpy as np
import statsmodels.formula.api as smf

## Data

In [4]:
from transplant.data.learningset import Learningset

learningset = Learningset()

X_train, X_test = learningset.get_data_merged_dynamic_flatten_full(full_df=True)
#X_train, X_test, y_train, y_test, X_col = learningset.get_data_merged_dynamic_flatten_full()

In [5]:
#X_train.dtypes

J'ai eu des petits problèmes avec le format des noms ( ne supporte pas les espaces, les points et les parenthèses )

In [6]:
X_train=X_train.rename(columns = {'BIS SR_mean':'BIS_SR_mean','B.I.S_mean':'BIS_mean','SvO2 (m)_mean':'SvO2m_mean',
                                  'BIS SR_std':'BIS_SR_std','B.I.S_std':'BIS_std','SvO2 (m)_std':'SvO2m_std',
                                 'BIS SR_max':'BIS_SR_max','B.I.S_max':'BIS_max','SvO2 (m)_max':'SvO2m_max',
                                 'BIS SR_min':'BIS_SR_min','B.I.S_min':'BIS_min','SvO2 (m)_min':'SvO2m_min'})

In [7]:
X_col= X_train.columns.drop('target')

## Feature selection for test

Ce n'est peut être pas une bonne pratique de mettre toutes les variables d'un coup mais on peut essayer

In [8]:
liste_full=X_col[0]
for i in X_col[1:] :
    liste_full+=' + ' + i

results = smf.ols('target ~ '+liste_full, data=X_train).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.634
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     1.439
Date:                Tue, 29 Jan 2019   Prob (F-statistic):             0.0284
Time:                        19:10:16   Log-Likelihood:                -48.619
No. Observations:                 228   AIC:                             347.2
Df Residuals:                     103   BIC:                             775.9
Df Model:                         124                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

#### Variables significatives au niveau 10 %

On pourrait essayer des modèles avec ça 

In [9]:
results.pvalues[results.pvalues<0.1]

Age_donor                     0.031160
LAS                           0.005837
RX_donor                      0.003209
pathologie                    0.065601
preoperative_ICU              0.044134
sexe                          0.078474
super_urgence                 0.022341
FC_mean                       0.093040
FR_mean                       0.053042
PEEPtotal_mean                0.063501
Pmean_mean                    0.089701
SvO2m_mean                    0.066594
VT_mean                       0.006718
declampage_cote2_done_mean    0.019091
BIS_std                       0.033348
FR_std                        0.006837
Pmean_std                     0.068101
SvO2m_std                     0.007582
VT_std                        0.003509
declampage_cote2_done_std     0.026748
BIS_max                       0.015943
DC_max                        0.016897
PNId_max                      0.060610
SvO2m_max                     0.017522
Temp_max                      0.056340
BIS_SR_min               

## Selections plus fines 

A poursuivre ...

In [10]:
liste_10_premiers_static=X_train.columns[0]
for i in X_train.columns[1:10] :
    liste_10_premiers_static+=' + ' + i
liste_10_premiers_static

'Age_donor + Aspirations_donor + BMI_donor + Donneur_CPT + Insuffisance_renale + LAS + PAPS + PFO + PF_donor + Poids'

In [11]:
results = smf.ols('target ~ '+liste_10_premiers_static, data=X_train).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     2.160
Date:                Tue, 29 Jan 2019   Prob (F-statistic):             0.0214
Time:                        19:10:16   Log-Likelihood:                -152.40
No. Observations:                 228   AIC:                             326.8
Df Residuals:                     217   BIC:                             364.5
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.7855    